In [1]:
import importlib
from IPython.display import clear_output

!pip install yfinance
clear_output()
!pip install yahoofinancials
clear_output()
!pip install yahoo-finance
clear_output()
!pip install yahoo_fin
clear_output()
!pip install yahoo_fin --upgrade
clear_output()
!pip install requests_html
clear_output()
!pip install gspread
clear_output()
!pip install XlsxWriter
clear_output()


# Install package to use outside ipynb files
ipynb_spec = importlib.util.find_spec("ipynb")
found = ipynb_spec is not None
if (found != True):
  !pip install import-ipynb

# Import ipynb imports
import import_ipynb

# Clear the outputs of the previous stuff
clear_output()

# Open notebook location and set to directory
from google.colab import drive
drive.mount('/content/drive')
import sys
from os.path import dirname
sys.path.append('/content/drive/My Drive/Colab Notebooks/')

# Setting to None on first run
historical_datas = None


# install stuff to access directory
kora_spec = importlib.util.find_spec("kora")
found = kora_spec is not None
if (found != True):
  !pip install kora -q

# Import kora
from kora import drive
drive.link_nbs()



# Import functions for stocks
import ScreenerOptions as scrOpt
import OutputOptions as outOpt

# Clear the outputs of the previous stuff
clear_output()



In [2]:
# Mount the drive and import packages
from yahoofinancials import YahooFinancials
from yahoo_fin.stock_info import get_data
from yahoo_finance import Share
import pandas as pd
import numpy as np
import math
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def getHistoricalData(initial_df):
  # Imports
  from yahoofinancials import YahooFinancials
  from yahoo_fin.stock_info import get_data
  from yahoo_finance import Share
  import pandas as pd
  import numpy as np

  # Get Data
  ticker_list = initial_df['Symbol']
  historical_datas = {}
  failedList = []
  for ticker in ticker_list:
      try:
        historical_datas[ticker] = get_data(ticker, start_date = '01/01/2020' , end_date = '12/31/2020')
        #print(f"saved {ticker}")
      except:
        failedList.append(ticker)


  # Clear the outputs of the previous stuff
  from IPython.display import clear_output
  clear_output()

  
  return historical_datas

In [4]:
master_df = pd.read_excel('/content/drive/MyDrive/StocksToScreen/StockMasterList.xlsx')
test_df = master_df.copy()

# Testing 
# historical_data_test = getHistoricalData(test_df)

# Data Gathering Functions

In [8]:
def getTrailing12MonthsPriceData(ticker_list):
  # Get Dates
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date
  # Get data for one year

  todayD = date.today()
  offset = BMonthEnd(-1)
  offsetFirst = BMonthBegin(-13)

  # This will be last day of the previous month (this is for FIP not 2 week CR)
  #lastDayLastMonth = todayD+offset

  # This will be one year ago for current month
  firstDayFirstMonth = todayD+offsetFirst

  # Create new data frame to hold and start building a complete data frame from all the stocks on the list
  full_df = pd.DataFrame()
  tickerWithNotEnoughData = []

  for ticker in ticker_list:
    try:
      # Here we are getting historical data from one year ago until today.
      full_df[ticker] = historical_data_test[ticker]['close'].loc[f"{firstDayFirstMonth.year}-{firstDayFirstMonth.month}-{firstDayFirstMonth.day}" : f"{todayD.year}-{todayD.month}-{todayD.day}"]
    except:
      tickerWithNotEnoughData.append(ticker)

  return full_df

In [9]:
def getTrailing12MonthsVolumeData(ticker_list):
  # Get Dates
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date
  # Get data for one year

  todayD = date.today()
  offset = BMonthEnd(-1)
  offsetFirst = BMonthBegin(-13)

  # This will be last day of the previous month
  #lastDayLastMonth = todayD+offset

  # This will be one year ago for current month
  firstDayFirstMonth = todayD+offsetFirst

  # Create new data frame to hold and start building a complete data frame from all the stocks on the list
  full_df = pd.DataFrame()
  tickerWithNotEnoughData = []

  for ticker in ticker_list:
    try:
      # Here we are getting historical data from one year ago until today.
      full_df[ticker] = historical_data_test[ticker]['volume'].loc[f"{firstDayFirstMonth.year}-{firstDayFirstMonth.month}-{firstDayFirstMonth.day}" : f"{todayD.year}-{todayD.month}-{todayD.day}"]
    except:
      tickerWithNotEnoughData.append(ticker)

  
  return full_df

In [10]:
def getYearlyVolumeData(ticker_list, desiredYear):
  # Get Dates
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  import datetime
  # Get data for one year

  # First day of Year
  firstDayYear = datetime.datetime(desiredYear, 1, 1)
  # Last day of Year
  lastDayYear = datetime.datetime(desiredYear, 12, 31)

  # Create new data frame to hold and start building a complete data frame from all the stocks on the list
  volume_df = pd.DataFrame()
  tickerWithNotEnoughData = []

  for ticker in ticker_list:
    try:
      # Here we are getting historical data from one year ago until today.
      volume_df[ticker] = historical_data_test[ticker]['volume'].loc[f"{firstDayYear.year}-{firstDayYear.month}-{firstDayYear.day}" : f"{lastDayYear.year}-{lastDayYear.month}-{lastDayYear.day}"]
    except:
      tickerWithNotEnoughData.append(ticker)
  return volume_df

In [11]:
def getYearlyPriceData(ticker_list, desiredYear):
  # Get Dates
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  import datetime
  # Get data for one year



  # First Day Year
  firstDayYear = datetime.datetime(desiredYear, 1, 1)

  # Last Day Year
  lastDayYear = datetime.datetime(desiredYear, 12, 31)

  # Create new data frame to hold and start building a complete data frame from all the stocks on the list
  close_df = pd.DataFrame()
  tickerWithNotEnoughData = []

  for ticker in ticker_list:
    try:
      # Here we are getting historical data from one year ago until today.
      close_df[ticker] = historical_data_test[ticker]['close'].loc[f"{firstDayYear.year}-{firstDayYear.month}-{firstDayYear.day}" : f"{lastDayYear.year}-{lastDayYear.month}-{lastDayYear.day}"]
    except:
      tickerWithNotEnoughData.append(ticker)
  return close_df

In [12]:
volume_History_2020 = getYearlyVolumeData(test_df['Symbol'].to_list(), 2020)
price_History_2020 = getYearlyPriceData(test_df['Symbol'].to_list(), 2020)

volume_History_2019 = getYearlyVolumeData(test_df['Symbol'].to_list(), 2019)
price_History_2019 = getYearlyPriceData(test_df['Symbol'].to_list(), 2019)

volume_History_2018 = getYearlyVolumeData(test_df['Symbol'].to_list(), 2018)
price_History_2018 = getYearlyPriceData(test_df['Symbol'].to_list(), 2018)

volume_History_2017 = getYearlyVolumeData(test_df['Symbol'].to_list(), 2017)
price_History_2017 = getYearlyPriceData(test_df['Symbol'].to_list(), 2017)

volume_History_2016 = getYearlyVolumeData(test_df['Symbol'].to_list(), 2016)
price_History_2016 = getYearlyPriceData(test_df['Symbol'].to_list(), 2016)

In [ ]:
# volume_History_2020
# price_History_2020

# volume_History_2019
# price_History_2019

# volume_History_2018
# price_History_2018

# volume_History_2017
# price_History_2017

# volume_History_2016
# price_History_2016

# Testing Excel outputs

In [13]:
volume_History_2020.to_excel("Master_Volume_History_2020.xlsx", index=True)
price_History_2020.to_excel("Master_Price_History_2020.xlsx", index=True)

volume_History_2019.to_excel("Master_Volume_History_2019.xlsx", index=True)
price_History_2019.to_excel("Master_Price_History_2019.xlsx", index=True)

volume_History_2018.to_excel("Master_Volume_History_2018.xlsx", index=True)
price_History_2018.to_excel("Master_Price_History_2018.xlsx", index=True)

volume_History_2017.to_excel("Master_Volume_History_2017.xlsx", index=True)
price_History_2017.to_excel("Master_Price_History_2017.xlsx", index=True)

volume_History_2016.to_excel("Master_Volume_History_2016.xlsx", index=True)
price_History_2016.to_excel("Master_Price_History_2016.xlsx", index=True)


In [16]:
  from pandas.tseries.offsets import BMonthEnd, BMonthBegin, DateOffset
  from datetime import date

  test_previous_Dates = date.today()
  print(test_previous_Dates)

2021-05-05


In [15]:
# Getting a master list for Quality
halfList = int(len(master_df['Symbol'].to_list()) / 2.0)
master_df_copy_top = master_df.head(halfList)
master_df_copy_bottom = master_df_copy.tail(halfList)



In [1]:
print(master_df_top)

NameError: ignored

In [16]:
# Getting the top half of the master list
master_quality_df_topHalf = outOpt.QualityOutput(master_df_copy_top)

Failure Count in Quality: 10510
     Symbol  ...       mktCap
0         A  ...  40573583360
1        AA  ...   7552360448
2       AAL  ...  13863492608
3      AAME  ...     85335536
4       AAN  ...   1114276096
...     ...  ...          ...
1730     JG  ...    392650272
1731    JGH  ...    368057120
1732    JHB  ...    525739488
1733    JHG  ...   6378673152
1734    JHX  ...  14900768768

[1735 rows x 15 columns]


In [17]:
master_quality_df_topHalf.to_excel('Top_Half_Master_Quality_List.xlsx',index=True)

In [18]:
master_quality_df_botHalf = outOpt.QualityOutput(master_df_copy_bottom)

Failure Count in Quality: 10501
     Symbol  ...       mktCap
0         A  ...  40637566976
1        AA  ...   7568204800
2       AAL  ...  13834630144
3      AAME  ...     83294016
4       AAN  ...   1113198080
...     ...  ...          ...
1730     JG  ...    385575520
1731    JGH  ...    367825344
1732    JHB  ...    525459968
1733    JHG  ...   6375226880
1734    JHX  ...  14918775808

[1735 rows x 15 columns]


In [ ]:
to_excel('Top_Half_Master_Quality_List.xlsx',index=True)